# Ejercicio 6: Dense Retrieval e Introducción a FAISS

## Objetivo de la práctica

Generar embeddings con sentence-transformers (SBERT, E5), e indexar documentos con FAISS 

## Parte 0: Carga del Corpus
### Actividad

1. Carga el corpus 20 Newsgroups desde sklearn.datasets.fetch_20newsgroups.
2. Limita el corpus a los primeros 2000 documentos para facilitar el procesamiento.

In [2]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

# Cargar el corpus 20 Newsgroups
print("Cargando corpus 20 Newsgroups...")
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# Limitar a los primeros 2000 documentos
documents = newsgroups.data[:2000]
print(f"Total de documentos cargados: {len(documents)}")
print(f"Ejemplo de documento:\n{documents[0][:200]}...\n")

Cargando corpus 20 Newsgroups...
Total de documentos cargados: 2000
Ejemplo de documento:


I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However,...



## Parte 2: Generación de Embeddings
### Actividad

1. Usa dos modelos de sentence-transformers. Puedes usar: `'all-MiniLM-L6-v2'` (SBERT), o `'intfloat/e5-base'` (E5). Cuando uses E5, antepon `"passage: "` a cada documento antes de codificar.
2. Genera los vectores de embeddings para todos los documentos usando el modelo seleccionado.
3. Guarda los embeddings en un array de NumPy para su posterior indexación.

In [6]:
!pip install sentence-transformers

In [8]:
# Opción 1: Usando SBERT (all-MiniLM-L6-v2)
from sentence_transformers import SentenceTransformer
print("Cargando modelo SBERT: all-MiniLM-L6-v2...")
model_sbert = SentenceTransformer('all-MiniLM-L6-v2')

print("Generando embeddings con SBERT...")
doc_embeddings_sbert = model_sbert.encode(documents, show_progress_bar=True)
print(f"Dimensión de embeddings SBERT: {doc_embeddings_sbert.shape}")

Cargando modelo SBERT: all-MiniLM-L6-v2...
Generando embeddings con SBERT...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Dimensión de embeddings SBERT: (2000, 384)


In [9]:
# Guardar embeddings en arrays de NumPy
np.save('embeddings_sbert.npy', doc_embeddings_sbert)
print("\nEmbeddings guardados exitosamente")


Embeddings guardados exitosamente


In [ ]:
# Opción 2: Usando E5 (intfloat/e5-base)
print("\nCargando modelo E5: intfloat/e5-base...")
model_e5 = SentenceTransformer('intfloat/e5-base')
 
# Preprocesar documentos con prefijo "passage: " para E5
documents_e5 = ["passage: " + doc for doc in documents]

print("Generando embeddings con E5...")
doc_embeddings_e5 = model_e5.encode(documents_e5, show_progress_bar=True)
print(f"Dimensión de embeddings E5: {doc_embeddings_e5.shape}")

# Guardar embeddings en arrays de NumPy
np.save('embeddings_e5.npy', doc_embeddings_e5)
print("\nEmbeddings guardados exitosamente")

## Parte 3: Indexación con FAISS
### Actividad

1. Crea un índice plano con faiss.IndexFlatL2 para búsquedas por distancia euclidiana.
2. Asegúrate de usar la dimensión correcta `(embedding_dim = doc_embeddings.shape[1])`.
3. Agrega los vectores de documentos al índice.

## Parte 4: Consulta Semántica
### Actividad

1. Escribe una consulta en lenguaje natural. Ejemplos:

    * "God, religion, and spirituality"
    * "space exploration"
    * "car maintenance"

2. Codifica la consulta utilizando el mismo modelo de embeddings. Cuando uses E5, antepon `"query: "` a la consulta.
3. Recupera los 5 documentos más relevantes con `index.search(...)`.
4. Muestra los textos de los documentos recuperados (puedes mostrar solo los primeros 500 caracteres de cada uno).

In [10]:
def search_documents(query, model, doc_embeddings, documents, top_k=5, use_e5=False):
    # Codificar la consulta
    if use_e5:
        query_text = "query: " + query
    else:
        query_text = query
    
    query_embedding = model.encode([query_text])
    
    # Calcular similitud coseno
    similarities = cosine_similarity(query_embedding, doc_embeddings)[0]
    
    # Obtener los índices de los top_k documentos más similares
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    return top_indices, similarities[top_indices]

In [12]:
queries = [
    "God, religion, and spirituality",
    "space exploration",
    "car maintenance"
]

print("\n" + "="*70)
print("BÚSQUEDAS CON SBERT (all-MiniLM-L6-v2)")
print("="*70)

for query in queries:
    print(f"CONSULTA: '{query}'")
    
    indices, scores = search_documents(
        query, 
        model_sbert, 
        doc_embeddings_sbert, 
        documents, 
        top_k=5
    )
    
    for rank, (idx, score) in enumerate(zip(indices, scores), 1):
        print(f"Resultado #{rank} (Similitud: {score:.4f})")
        print(f"Documento #{idx}")
        print(f"{documents[idx][:500]}...")
        print(f"{'-'*70}\n")






BÚSQUEDAS CON SBERT (all-MiniLM-L6-v2)
CONSULTA: 'God, religion, and spirituality'
Resultado #1 (Similitud: 0.4150)
Documento #996




Humanist, or sub-humanist? :-)...
----------------------------------------------------------------------

Resultado #2 (Similitud: 0.3307)
Documento #282

I didn't know God was a secular humanist...

Kent...
----------------------------------------------------------------------

Resultado #3 (Similitud: 0.3013)
Documento #677
 
(Deletion)
 
For me, it is a "I believe no gods exist" and a "I don't believe gods exist".
 
In other words, I think that statements like gods are or somehow interfere
with this world are false or meaningless. In Ontology, one can fairly
conclude that when "A exist" is meaningless A does not exist. Under the
Pragmatic definition of truth, "A exists" is meaningless makes A exist
even logically false.
 
A problem with such statements is that one can't disprove a subjective god
by definition, and...
--------------------------------

In [ ]:
print("\n" + "="*70)
print("BÚSQUEDAS CON E5 (intfloat/e5-base)")
print("="*70)

for query in queries:
    print(f"\n{'='*70}")
    print(f"CONSULTA: '{query}'")
    print(f"{'='*70}\n")
    
    indices, scores = search_documents(
        query, 
        model_e5, 
        doc_embeddings_e5, 
        documents, 
        top_k=5,
        use_e5=True
    )
    
    for rank, (idx, score) in enumerate(zip(indices, scores), 1):
        print(f"Resultado #{rank} (Similitud: {score:.4f})")
        print(f"Documento #{idx}")
        print(f"{documents[idx][:500]}...")
        print(f"{'-'*70}\n")

In [ ]:
# ============================================================
# Comparación de resultados
# ============================================================

print("\n" + "="*70)
print("ANÁLISIS COMPARATIVO")
print("="*70)

test_query = "space exploration"
print(f"\nComparando resultados para: '{test_query}'\n")

# SBERT
indices_sbert, scores_sbert = search_documents(
    test_query, model_sbert, doc_embeddings_sbert, documents, top_k=5
)

# E5
indices_e5, scores_e5 = search_documents(
    test_query, model_e5, doc_embeddings_e5, documents, top_k=5, use_e5=True
)

print("Top 5 con SBERT:")
for rank, (idx, score) in enumerate(zip(indices_sbert, scores_sbert), 1):
    print(f"  {rank}. Doc #{idx} - Similitud: {score:.4f}")

print("\nTop 5 con E5:")
for rank, (idx, score) in enumerate(zip(indices_e5, scores_e5), 1):
    print(f"  {rank}. Doc #{idx} - Similitud: {score:.4f}")

# Calcular overlap
overlap = len(set(indices_sbert) & set(indices_e5))
print(f"\nDocumentos en común: {overlap}/5")